In [ ]:
#fmain
import os
import pandas as pd
import pprint

from matplotlib import cm
import pycircos

import fconfig as cfg

DBG = False

In [ ]:
def get_matplot_colors(colormap_name,num):
    color_dict = {}
    color_map = cm.get_cmap(colormap_name, num)
    for i in range(color_map.N):
        color_dict[i] = cm.colors.rgb2hex(color_map(i))
    return color_dict
# get_matplot_colors('rainbow',15) # test


In [ ]:
class DataList:
    def __init__(self, dfs, sheet, sheet_matrix ):
        self.sheet = sheet 
        self.parts_skip = []

        # Get the dataframe for this sheet
        #   sort by Partition (row=2) and then Network (row=0)
        #   ex: Blast - Sort by '751 blast' 
        #   must drop Column 0 bc it contains strings and sort can only sort same type
        df = dfs[self.sheet]
        df = df.drop(columns=[0])
        self.sheet_df = df.sort_values(by=[2,0], axis=1)

        # create sorted file for debugging
        self.create_sorted_csv()

        # 0,1,2 are rows to read in as a list, then remove index at col=0 
        self.netw_list = list(self.sheet_df.to_records()[0])[1:] 
        self.nodes_list = list(self.sheet_df.to_records()[1])[1:] 
        self.part_list = list(self.sheet_df.to_records()[2])[1:] 

        # These 2 dicts hold all data
        self.matrix_node_map = self.read_matrix_sheet( sheet_matrix ) 
        self.part_map = self.create_partition_map()

        # add data from matrix sheet 
        self.add_nodes_to_part_map()

        #??
        self.map_of_nodes = self.create_map_of_nodes()

        # circos cant handle these conditions
        self.validate_data()
        # BRITT _ USED VALIDATE TO CREATE NODE ROWS 
        self.create_net_rows()

        # self.create_node_rows()
        self.create_node_rows()

        # print("\n\n\n ARCS FOR PART 6====================")
        # pprint.pp( self.part_map[6])

    def create_map_of_nodes(self):
        ''' this is used for looking up ALL node informatio for partition '''
        # nodes: rel_col, abs_col, part, network
        d = {}
        for p in self.part_map:
            node_dict = self.part_map[p]['nodes_start_stop']
            for node in node_dict:
                d[node] = node_dict[node]
 
        # print("MAP NODES D",d)
        return d
    
    def create_sorted_csv(self):
        #  DEBUG, create sorted CSV
        filename = self.sheet+'_sorted.csv'
        self.sheet_df.to_csv( filename )
        print(f'\n{"Overwrite" if os.path.exists(filename) else "Created"}: {filename}')

    def create_partition_map(self):
        ''' This CONTAINS most all data necessary to draw diagrams'''
        if DBG:
            print('Uniq Partitions:',list(set(self.part_list)))

        part_list = self.part_list
        netw_list = self.netw_list
      
        pdict = self.build_start_stop( part_list,netw_list)

        # get network abs cols and put part in there for printint
        for p in pdict.keys():
            # pdict[p]['partid']=str(p) unnecessary
            pdict[p]['rel_nets'] = self.build_start_stop(pdict[p]['nets'],None )
            # pprint.pp( pdict )
            if pdict[p]['len']==1:
                self.parts_skip.append(p)

        if DBG:
            print("create_partition_map")
            pprint.pp(pdict )


        # add list of nodes to this partition 
  
        pdict_nodes = self.build_start_stop( part_list,self.nodes_list)
        # add nodes and list of rel_nodes to pdict - reuse build start stop
        for p in pdict.keys():
            pdict[p]['nodes']=pdict_nodes[p]['nets'] # reuse this key
            node_start_stop = self.build_start_stop(pdict_nodes[p]['nets'])
            # print(node_start_stop)
            pdict[p]['nodes_start_stop'] = node_start_stop

        # add Network for each Node into node_start_stop
        for p in pdict.keys():
            nets = pdict[p]['nets']
            nodes = pdict[p]['nodes']
            pdict[p]['nodes_start_stop']
            for ct,node in enumerate(nodes):
                pdict[p]['nodes_start_stop'][node]['netw'] = nets[ct]

                # add absolute col (used for debug info)
                part_col_start = pdict[p]['start']
                node_rel_start = pdict[p]['nodes_start_stop'][node]['start']
                # print(" calc node abs cols and rel col",part_col_start,node_rel_start )
                # ADD 1 to abs col bc cols are zero based 
                pdict[p]['nodes_start_stop'][node]['node_abs_col'] = part_col_start+node_rel_start+1
                pdict[p]['nodes_start_stop'][node]['part'] = p

                # add skip key - don't draw - fill reason in later
                pdict[p]['nodes_start_stop'][node]['skip'] =[]
        
        # add nodes into rel_nets - get nodes for networks
        for p in pdict.keys():
            node_ss = pdict[p]['nodes_start_stop']
            for node in node_ss.keys():
                netw = node_ss[node]['netw']
                if 'nodes' not in pdict[p]['rel_nets'][netw].keys():
                    pdict[p]['rel_nets'][netw]['nodes'] = []
                pdict[p]['rel_nets'][netw]['nodes'].append(node) 

        return pdict

    def read_matrix_sheet(self,matrix_sheet):
        ''' returns dict ofsrc node id: and desc node id.
            dest node id is the COL of the 1st NON-ZERO item'''
        # returns list of columns numbers of first non-zero node
        self.matrix_df = pd.DataFrame(pd.read_excel(cfg.Ginput_file,
                sheet_name=matrix_sheet,))
        
        node_dict = {}
   
        # print("TYPE OF REC", type(self.matrix_df.to_records()))
        for row in self.matrix_df.to_records():
            l = list(row)
            # print(l)
            node = int(l[1])
            # Keep this since holds place for nodes with no non-zero values
            node_dict[node] = { 'node' : node }
            # print("NODE", node, type(node))
      
            for c,val in enumerate(l[2:]): # vals start 2:
                # print(type(val)) # SOME ARE INT64 SOME ARE FLOAT64 
                if val > 0:
                    # print(f"node {node} VALE {val} COL {c}")
                    node_dict[ node ]= {
                        'dest_node' : c+1,
                        'val' : val,
                    }
                    break # go to next ROW
        # print("\n\n MATRIX DICT")
        # pprint.pp( node_dict)
     
        z = [ node for node,ninfo in node_dict.items() if 'dest_node' not in ninfo ]
        print(f"{matrix_sheet} Nodes with 0.0 values: ",z )

        return node_dict

    def add_nodes_to_part_map(self ):
        ''' add dest nodes to this partition '''
  
        for p in self.part_map:
            node_dict = self.part_map[p]['nodes_start_stop']
            for src_node in node_dict:
                dest_node_dict = self.matrix_node_map[src_node]
                if 'dest_node' in dest_node_dict:
                    dest_node = dest_node_dict['dest_node']
                    # print(f"Partition {p} Node {src_node} DestNodeX {dest_node}")
                    self.part_map[p]['nodes_start_stop'][src_node]['dest_node']=dest_node

      
    def validate_data(self):
        ''' circos cannot handle partitions of length one,
            or networks of length one.
            Add skip info all at once - must be run 
                BEFORE building NODE_ROWS '''
        print("\nvalidate_data()")

        self.skip_nodes = []
        for p in self.part_map:
            self.part_map[p].update(warn=[])
            self.part_map[p].update(skip_parts=[])
            self.part_map[p].update(skip_nets=[])
            self.part_map[p].update(skip_nodes=[])

            # check for partitions of length one
            if self.part_map[p]['len']==1:
                self.part_map[p]['warn'].append( f'Partition:{p} ONLY 1 Network' )
                self.part_map[p]['skip_parts'].append(p)
                # print("SKIP PARTs",self.part_map[p]['skip_parts'])
            
            # check for networks of length one
            for n in self.part_map[p]['rel_nets']: 
                rel_nets = self.part_map[p]['rel_nets']
                if len(nodes:=rel_nets[n]['nodes'])==1:
                    self.part_map[p]['warn'].append( f'Network:{n} contains ONLY 1 Node:{nodes[0]}' )
                    self.part_map[p]['skip_nets'].append(n)
            
            # Cases not to draw arcs:
            # a) node in matrix sheet has 0.0 values (no NON-ZERO values)
            # b) node in matrix sheet is the same as this node # BRITT TODO???
            # c) node has dest node in the same partition and network
            # 
            node_ss = self.part_map[p]['nodes_start_stop']
            for node,node_info in node_ss.items():
                # skip all nodes if partition not drawn
                if p in self.part_map[p]['skip_parts']:
                    self.part_map[p]['warn'].append(  f'Partition:{p} not Drawn, do not Draw its nodes {node}' )
                    self.part_map[p]['skip_nodes'].append(node)

                # node_info = node_ss[node]
                if 'dest_node' not in node_info:
                    self.part_map[p]['warn'].append( f'Node:{node} Matrix Sheet value 0.0' )
                    self.part_map[p]['skip_nodes'].append(node)
                    # node_info['warn'].append(('no_dest_node',node))
                else:
                    dest_node = node_info['dest_node']
                    if dest_node in node_ss:
                        dest_info = node_ss[dest_node]
                        # print("BRITT p=",p," node:",node,' nodekey',node_info.keys(),' dest_info',dest_info)
                        if 'dest_node' in dest_info:
                            dest = dest_info['dest_node']
                            if node == dest:
                                node_info['skip'].append(f'src=dest {node},{dest_node}')  
                                self.part_map[p]['warn'].append( f'Node:{node} == Matrix Sheet Node' )
                                self.part_map[p]['skip_nodes'].append(node)   

            self.skip_nodes += self.part_map[p]['skip_nodes']
        self.skip_nodes = sorted(self.skip_nodes)
        # print("sSKIP NODES",self.skip_nodes)

    def test_analyze_data(self):
        print(f"\n***{self.sheet}")

        skip_parts = [] 
        for p in self.part_map.keys():
            for sp in self.part_map[p]['skip_parts']:
                skip_parts.append( sp )
        
        draw_parts = [x for x in list(self.part_map.keys()) if x not in skip_parts]
           
        print(f"{len(self.part_map.keys())} Partitions: {list(self.part_map.keys())}")
        print(f"    {len(skip_parts)} Partitions not Drawn: {skip_parts}")
        print(f"    {len(draw_parts)} Partitions >1 Network: {draw_parts}")
        print(f"    {len(self.skip_nodes)} Nodes not Drawn: {self.skip_nodes}")
        self.get_part_rows(True)
        self.get_net_rows(True)
        self.get_node_rows(True)
        
        for p in self.part_map.keys():
            self.test_analyze_data_part(p)

    def test_analyze_data_part(self,p):
        ''' print analysis of Partition'''
        part = self.part_map[p]

        skip_nets = sorted(list(set(self.part_map[p]['skip_nets'])))
        nets = list(set(part['nets']))
        draw_nets = [x for x in nets if x not in skip_nets]

        skip_nodes = sorted(list(set(self.part_map[p]['skip_nodes'])))
        nodes = list(set(part['nodes_start_stop'].keys()))
        draw_nodes = [x for x in nodes if x not in skip_nodes]

        print(f"\nPartition: {p}")
        print(f"    Drawn: {self.draw_part(p)}")
        print(f"    {len(nets)} Networks: {nets}")    
        # print(f"        Networks containing 1 Node: {self.get_one_node_list(p)}")
        print(f"        {len(skip_nets)} Networks not Drawn: {skip_nets}")
        print(f"        {len(draw_nets)} Networks >1 Node: {draw_nets}")

        print(f"    {len(part['nodes'])} Nodes: {part['nodes']}")
        print(f"        {len(skip_nodes)} Nodes not Drawn: {skip_nodes}")
        print(f"        {len(draw_nodes)} Nodes     Drawn: {draw_nodes}")

        print(f"\n    Warn List: {part['warn']}")
       

    def test_show_part_info(self,part):
        print("=== All info for Partition: {part}")
        pprint.pp(self.part_map[part])

    #
    # THIS BUILDS DATA USED TO GENERATE DIAGRAMS
    def draw_part(self,p):
        return p not in self.part_map[p]['skip_parts']

    def get_part_rows(self,label=False):
        ''' str id, length, color '''
        ''''  row = (PART_ID (str), LENGTH(int), COLOR(hex string or name)'''
        # print(f"Partitions: #{len(self.part_map.keys())}")
        print(f"DRAW_PARTS_WITH_ONE_NODE={cfg.DRAW_PARTS_WITH_ONE_NODE}")
        color_dict = get_matplot_colors(cfg.PARTITION_COLORS,
                        len(self.part_map.keys())+1)
        rows = []
        for p in self.part_map.keys():
            # # only add row if len>1 or draw and len>1
            if self.draw_part(p) or cfg.DRAW_PARTS_WITH_ONE_NODE:
        
                color = color_dict[p] if not cfg.WHITE_PARTS else 'white'
                rows.append( (str(p), # must be string
                              self.part_map[p]['len'], #length
                              color) )
        if label:
            print(f"PARTITION PLOT DATA\nPartition, Length, Color")
            print(rows)
        return rows


    def create_net_rows(self):
        for p in self.part_map.keys():
            self.part_map[p].update(net_row=[])
            # print("Part SKip Nets",p,' skip=',self.part_map[p]['skip_nets'])
            rn = self.part_map[p]['rel_nets']
            for n in rn:
                if n not in self.part_map[p]['skip_nets']:
                    # SUBTRACT 1 from LEN !!! If not bar will extend past partitions
                    new_len = rn[n]['len']-1
                    self.part_map[p]['net_row'].append( (p, rn[n]['start'], #rn[n]['end'],
                                                    new_len, n ))

    def get_net_row_for_part(self,part):
        ''' return network rows for this partition'''
        # part_id, start, width, netw = row
        # if only 1 network then Divide by ZERO ERROR!!!
        return self.part_map[part]['net_row']
                
    def get_net_rows(self,label=False):
        # rows = [self.get_net_row_for_part(part) for part in self.part_map.keys()]
        rows = []
        for part in self.part_map.keys():
            rows = rows + self.get_net_row_for_part(part)
        if label:
            print(f"NETWORK PLOT DATA\nPartition, Network_Start_Col, Length, Network")
            print(rows)
        return rows


    def create_node_rows(self):
        ''' for p,s,sw,d,dw,n,net in rows:
            src = (str(p),s,sw,1000)
            dest = (str(p),d,dw,1000)
            '''
        CIRCLE_DIAM = 1000
        # add 'node_rows' into part_map 
        #
        print(f"- SKIP  NODES: {self.skip_nodes}")
        for p in self.part_map.keys():
            self.part_map[p].update(node_rows=[])
            nodes_dict= self.part_map[p]['nodes_start_stop']

            for node,node_info in nodes_dict.items():

                if node not in self.skip_nodes:
                
                    col = node_info['start']
                    node_len = node_info['len']
                    src = (str(p),col,node_len,CIRCLE_DIAM)

                    dest_node = node_info['dest_node']
                    # print(f"Check if DEST NODE in SKIP NODE LIST {dest_node}, {dest_node in self.skip_nodes}")
                    
                    if dest_node not in self.skip_nodes:
                        dest_node_info = self.map_of_nodes[dest_node]
                        # print("IN GENERTA ATC LIST",dest_node )

                        desc_col = dest_node_info['start'] 
                        dest_node_len = dest_node_info['len']
                        dest_p = dest_node_info['part']
                        dest = (str(dest_p),desc_col,dest_node_len,CIRCLE_DIAM)

                        self.part_map[p]['node_rows'].append( (src,dest,node_info['netw']))



    def get_node_rows(self,label=False):
        # rows = [self.part_map[part]['node_rows'] for part in self.part_map.keys() ]
        rows = []
        for part in self.part_map.keys():
            rows = rows + self.part_map[part]['node_rows']
        if label:
            print(f"{len(rows)} NODE PLOT DATA: Src,Dest (Part,Col,Length,Network,Diameter)")
            print(rows)
        return rows

    def build_start_stop(self,vals,nets=None): #BRITT USE DIC COMT
        d = {}
        ct = 0
        for v in vals:
            if v not in d.keys():
                d[v]={}
                d[v]['start'] = ct
                if nets:
                    d[v]['nets'] = []
            if nets:
                d[v]['nets'].append(nets[ct])
            d[v]['end']=ct 
            d[v]['len']=d[v]['end']-d[v]['start']+1
            ct+=1
        return d   



In [ ]:

class ReadExcelFile:
    def __init__(self,input_file):
        ''' Generate the csv files for this excel file.
            Notes: 
                1. header=None will read in ALL SPREADSHEETS
                with this setting.
                Matrix Spreadsheet HAS a header so must be read in separately.
                Sorting will Fail if there is a header for subject sheet
                
                2. Nonblast Sheetname has trailing SPACE in name.
        '''
        self.input_file = input_file

        self.dfs = pd.read_excel(
            input_file,
            sheet_name=None, # READS ALL WORSHEETS
            header=None)
        # sheets are in pairs, first is subject, second is matrix
        
        self.sheet_list = list(zip(list(self.dfs.keys())[::2],list(self.dfs.keys())[1::2]))
        print(f"Opened File: {input_file}\n Sheets:{self.sheet_list}")

        self.data_dict = {}

    def process_sheets(self):
        for sub_sheet, matrix_sheet in self.sheet_list:
            self.data_dict[sub_sheet] = DataList( self.dfs, sub_sheet, matrix_sheet)

    def plot_all(self): 
        for sheet in self.data_dict.keys():
            data_obj = self.data_dict[sheet]

            part_rows = data_obj.get_part_rows()
            net_rows = data_obj.get_net_rows()
            node_rows = data_obj.get_node_rows()

            cir2 = PlotParts(part_rows )
            cir2 = PlotNetworks(net_rows,cir2,cfg.unh_colors1)
            PlotChords(node_rows,cir2)
            cir2.figure.suptitle("\n\n"+sheet, fontsize=30)

    def Xdraw_all(self): # OLD GET RID OF
        for sheet in self.data_dict.keys():
            data_obj = self.data_dict[sheet]
            cir = ChordCircle(sheet)
            cir.plot_it( data_obj.part_map, data_obj.get_nodes_list())

    # 
    # TEST CODE
    #
    def test_process_sheet(self,sheet,matrix_sheet):
        print("***TEST process_sheet()",sheet,matrix_sheet)
        self.data_dict[sheet] = DataList( self.dfs, sheet, matrix_sheet)

    def test_analyze_data(self):
        print(f"\n\n{'*'*20}\n***Analyzing Data for Excel File: {self.input_file}")
        print(f"Sheets:{self.sheet_list}")
        for sheet in self.data_dict.keys():
            self.data_dict[sheet].test_analyze_data()
    
    def test_analyze_data_part(self,sheet,part):
        ''' show detail for just this partition '''
        self.data_dict[sheet].test_analyze_data_part(part)
    
    def test_print_all_part_info(self,sheet,part):
        print(f"+++ ALL PARTITION INFO FOR Sheet: {sheet} Partition: {part}")
        pprint.pp(self.data_dict[sheet].part_map[part])
   
    def test_get_part_rows(self,sheet):
        ''' show rows sent to diagram '''
        return self.data_dict[sheet].get_part_rows()
   
    def test_get_net_rows(self,sheet):
        ''' show rows sent to diagram '''
        return self.data_dict[sheet].get_net_rows()

    def test_get_net_rows_for_part(self,sheet,part):
        ''' return net rows for this sheet and partition'''
        return self.data_dict[sheet].get_net_row_for_part(part)

    def test_get_node_rows(self,sheet):
        return self.data_dict[sheet].get_node_rows()
    
        

In [ ]:
# THis code is in test_main.ipynb - used for testing too
def PlotParts(rows,interspace=cfg.INTERSPACE):
    '''  partid, length, color_name = row '''
    # plot circle
    circ = pycircos.Gcircle() 
    for row in rows:
        partid,length,color = row
        # if length>1:
        arcc = pycircos.Garc(
                arc_id=partid, 
                size=length, 
                facecolor=color,
                interspace=interspace, 
                raxis_range=(950,1000), 
                labelsize=20,labelposition=60, label_visible=True)
        circ.add_garc(arcc) 
    circ.set_garcs(0,360) # this displays the circle
    return circ  


def PlotNetworks(rows, circ,color_dict ):
    # WARNING!!!! must pass in circ, if store self.circ will NOT DRAW
    ''' REQUIRES: 
                part,
                number of networks, 
                network start col within part
                width of network
                network id = color
    '''
    for row in rows:
        part, start, width, netw = row
        circ.barplot( 
                str(part), # partition is string!
                data = [1], # number to draw (only drawing 1 at a time)
                positions = [start], # col WITHIN PARTITION (NOT CIRCLE COL)
                width = [width],
                facecolor = [ color_dict[netw] ],

                raxis_range=[950,1000], 
            )
    return circ
def PlotNetworksColor(rows, circ ):
    ''' see above for defn. This uses passed in colors'''
    # WARNING!!!! must pass in circ, if store self.circ will NOT DRAW
    for row in rows:
        part, start, width, color = row
        circ.barplot( 
                str(part), # partition is string!
                data = [1], # number to draw (only drawing 1 at a time)
                positions = [start], # col WITHIN PARTITION (NOT CIRCLE COL)
                width = [width],
                facecolor = color,
                raxis_range=[950,1000], 
            )
    return circ

def PlotNetworksV(parts_list,starts_list, width_list, color_list, circ ):
    # WARNING!!!! must pass in circ, if store self.circ will NOT DRAW
    ''' REQUIRES: 
                part,
                number of networks, 
                network start col within part
                width of network
                network id = color
    '''
    # THIS CREATS ALL NETWORKS AT ONCE FOR EACH PARTITION - instead of one net row at atime
    ct=0
    for part in parts_list:
        circ.barplot( 
                str(part), # partition is string!
                data = [1]*len(starts_list[ct]), # number to draw (only drawing 1 at a time)
                positions = starts_list[ct], # col WITHIN PARTITION (NOT CIRCLE COL)
                width = width_list[ct],
                facecolor = color_list[ct],

                raxis_range=[950,1000], 
            )
        ct+=1
    return circ

def PlotChords(rows,circ):
    ''' if part is specified, then only plot that partitions chords'''
    print(f"PlotChords() {len(rows)} Chords")
    # pprint.pp(rows) #BRITT
  
    for row in rows:
        src,dest,netw = row
        # print("CHORD src/dest,src,dest)
        color = cfg.unh_colors1[netw] if not cfg.BLACK_LINES else 'black'
  
        try:
            circ.chord_plot(src, dest, facecolor=color)
        except ZeroDivisionError:
            print(f"ERROR in PlotChords() ZeroDivisionError {src},{dest},{color} row=\n")
            print(row)
            print(f" Src,Dest (Part,Col,Length,Network,Diameter)")
        except Exception as e:
            print(f"ERROR in PlotChords() {e}")
            print(f" Src,Dest (Part,Col,Length,Network,Diameter)")
            print(f"src={src},dest={dest},netw={netw} color={color}")
    return circ

In [ ]:
def official_run():
    obj = ReadExcelFile( cfg.Ginput_file )
    obj.process_sheets()
    # obj.analyze_data() # SHOWS ALL!!
    obj.plot_all()

# official_run()

In [ ]:
# TEST CODE FOR CLEANUP AND BUG on Controls sheet Part 20
sheet='Controls'
matrix_sheet='Control matrix'
# sheet='Nonblast '
# matrix_sheet='Nonblast matrix'

# sheet='Blast'
# matrix_sheet='Blast matrix'
eobj = ReadExcelFile( cfg.Ginput_file )
eobj.test_process_sheet(sheet,matrix_sheet)
eobj.test_analyze_data_part(sheet,12)
eobj.plot_all()


# eobj.test_analyze_data()
# eobj.test_print_all_part_info(sheet,1) #BRITT

In [ ]:
# BRITT - TEST to read in Excel files 5/9/22 !!!
# 
def test_read_excel(sheet,matrix_sheet):
    eobj = ReadExcelFile( cfg.Ginput_file )
    eobj.process_sheet(sheet,matrix_sheet) 
    # obj.analyze_data() # SHOWS ALL!!

    part_rows = eobj.get_part_rows(sheet)
    net_rows = eobj.get_net_rows(sheet)
    node_rows = eobj.get_node_rows(sheet)


    cir2 = PlotParts(part_rows )
    cir2 = PlotNetworks(net_rows,cir2,cfg.unh_colors1)
    PlotChords(node_rows,cir2)

    p=3
    eobj.analyze_data_part(sheet,p)
    eobj.print_all_part_info(sheet,p)

# sheet='Controls'
# matrix_sheet='Control matrix'
# test_read_excel(sheet,matrix_sheet)

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

def show_color_bar(colors,label,tick_labels):
    cmap= mpl.colors.ListedColormap(colors)

    # one more and make ''
    bounds = tick_labels+[ tick_labels[-1]+1]

    # used to draw ticks
    norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

    # defines the width of the entire plot - 10 is wider
    fig, ax = plt.subplots(figsize=(20, 1))
    ax.tick_params(labelsize=40)
    
    #  defines the height of the entire plot
    # fig.subplots_adjust(bottom=0.5) # this makes it 1/2 the height
    cb = fig.colorbar(
        mpl.cm.ScalarMappable(cmap=cmap,norm=norm),
        cax=ax,
        orientation='horizontal',
        label=label,
    )
    # fig.suptitle('test title', fontsize=20) # is ontop of color bar
    plt.show()

In [ ]:
# https://matplotlib.org/3.5.0/gallery/color/named_colors.html
def plot_colorbar():
    import fconfig as cfg

    color_names = list(cfg.unh_colors1.values())
    color_nums = list( cfg.unh_colors1.keys())
    show_color_bar(color_names,'Network Colors',color_nums)
# plot_colorbar()

In [ ]:

d = {
    '1': { 'len' : 2 },
    '2': { 'len' : 1 }
}
# pprint.pp( d) 
# d.update(newkey1 ='portal') 
# pprint.pp( d) 
l1=[1,2,3,4]
l2=[3]
[x for x in l1 if x not in l2]